In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("InventoryAnalysis").getOrCreate()

cleaned_delta_file_path = "/delta/product_inventory_cleaned"

try:
    inventory_df = spark.read.format("delta").load(cleaned_delta_file_path)

    print("Cleaned Product Inventory Data:")
    inventory_df.show()

    inventory_analysis_df = inventory_df.withColumn(
        "TotalStockValue",F.col("StockQuantity") * F.col("Price"))

    restocking_df = inventory_analysis_df.filter(F.col(StockQuantity) < 100)

    print("Inventory Analysis Results:")
    inventory_analysis_df.show()
    
    print("Products Needing Restocking:")
    restocking_df.show()

    inventory_analysis_df.write.format("delta").mode("overwrite").save("/delta/inventory_analysis")
    print("Inventory analysis results saved successfully")

except AnalysisException:
    print("Cleaned Delta table not found")
except Exception as e:
    print(f"An error occurred: {str(e)}")
